---
title: Data Lake (DuckDB)
author: Jesús LM
date: 2025-08-12
date-modified: last-modified
date-format: '*MMM, YYYY*'
lang: es
format:
  revealjs:
    fig-width: 15
    brand: "_brand.yml"
    footer: "© 2025"
    embed-resources: true
    transition: convex
    transition-speed: slow
    title-slide-attributes:
      data-background-image: ../images/slides-main.png
      data-background-size: cover
      data-background-opacity: '0.9'
    mermaid:
      theme: flatly
jupyter: python3
---

## Diagrama de Flujo {background-image="../images/slides-secondary.png"}

::: {#fig-simple width=60%}

```{mermaid}
flowchart LR
    id1[Catalogo de datos]
    subgraph Capa de ingesta y transformación
    A["Archivos origen
    CSV, Excel, API, JSON"] -- "EL" --> B["Archivos parquet"]
    end
    subgraph Capa curada
    B -- T --> C[("Base de Datos
    (DuckDB)")]
    B -- T --> D[("Archivos no
    estructurados")]
    end
    subgraph Usuario final
    C --> E["Interfaz de
    descarga"]
    end
```
Diagrama del Data Lake

:::

![](./datalake2.png)


## Pasos {.smaller background-image="../images/slides-secondary.png"}

El proceso tiene contemplado los siguientes pasos:

* Crear un catalogo de datos
* Cargar los archivos origen en el repositorio del dueño de los datos
* Extraer y cargar los archivos origen a formato parquet
* Crear vistas SQL aplicando las transformaciones necesarias
* Descargar las vistas finales desde una interfaz gráfica

## ¿Por qué DuckDB? {.smaller background-image="../images/slides-secondary.png"}

DuckDB es una excelente opción para la creación de lagos de datos, ya que es una base de datos analítica, integrada y sin servidor que puede consultar archivos directamente en su sistema de archivos. Ofrece la potencia de un almacén de datos de alto rendimiento en un paquete ligero de un solo archivo.

1. Arquitectura integrada y sin servidor
2. Almacenamiento en columnas de alto rendimiento
3. Consulta directa de archivos
4. SQL enriquecido y funciones analíticas

## ¿Por qué Parquet? {.smaller background-image="../images/slides-secondary.png"}

El formato Parquet ofrece un rendimiento y una eficiencia mucho mayores para el análisis.

- **Almacenamiento en columnas.** Parquet es un formato de datos en columnas, lo que significa que almacena los datos por columna en lugar de por fila. DuckDB también es una base de datos columnar. Al ejecutar una consulta que solo necesita unas pocas columnas, DuckDB puede leer únicamente esas columnas del archivo Parquet, lo que resulta mucho más rápido que escanear un archivo CSV o JSON completo.
- **Compresión óptima.** Los archivos Parquet están altamente comprimidos, lo que permite ahorrar una cantidad significativa de espacio en disco.
- **Reconocimiento de esquemas.** Los archivos Parquet tienen un esquema integrado que ayuda a prevenir problemas con los tipos de datos y aumenta la fiabilidad de la carga de datos.

## ¿Por qué Vistas SQL? {.smaller background-image="../images/slides-secondary.png"}

La razón principal es que `CREATE VIEW` no almacena físicamente los datos dentro de la base de datos de DuckDB.
En su lugar, crea una tabla virtual (Los datos permanecen en sus archivos originales).

#### Beneficios

- **Sin duplicación de datos.** No se crea una copia de los datos CSV, JSON o Excel. Esto ahorra espacio en disco.
- **Siempre actualizado.** Si se modifica el archivo users.csv, la vista de usuarios reflejará esos cambios inmediatamente la próxima vez que se consulte. Una tabla creada con CREATE TABLE sería una instantánea estática y no se actualizaría automáticamente.
- **Flexibilidad.** Se pueden definir vistas para diferentes propósitos en los mismos archivos sin procesar sin tener que transformar y guardar varias tablas físicas.

## Catálogo de Datos {.smaller background-image="../images/slides-secondary.png"}

- **Descubrimiento y Acceso a los Datos**. Permite buscar y filtrar conjuntos de datos fácilmente, utilizando metadatos como descripciones, etiquetas o términos comerciales.

- **Gobernanza y Confianza en los Datos**. Puedes rastrear de dónde provienen los datos (linaje), quién es el dueño, quién tiene acceso y cómo se están utilizando. Esto genera confianza en la calidad y fiabilidad de los datos que se usan para la toma de decisiones.

- **Mejor Comprensión y Colaboración**. El catálogo de datos también puede incluir un glosario de términos de negocio, lo que asegura que todos en la organización tengan una comprensión común del significado de los datos.

## Ejemplo del catálogo {.smaller background-image="../images/slides-secondary.png"}
```{json}
[
  {
    "asset_name": "Estado de Fuerza Policial por Entidad Federativa, 2025",
    "technical_metadata": {
      "table_name": "fuerza_policial_2025",
      "source_system": "Secretaría de Seguridad y Protección Ciudadana",
      "schema": {
        "columns": [
          {
            "column_name": "entidad_federativa",
            "data_type": "VARCHAR(255)",
            "is_nullable": false,
            "description": "El nombre de la entidad federativa (de México."
          },
          {
            "column_name": "total_oficiales",
            "data_type": "INT",
            "is_nullable": false,
            "description": "El número total de oficiales de policía en la entidad."
          },
          {
            "column_name": "oficiales_municipales",
            "data_type": "INT",
            "is_nullable": false,
            "description": "Número de oficiales de policía a nivel municipal."
          },
          {
            "column_name": "oficiales_estatales",
            "data_type": "INT",
            "is_nullable": false,
            "description": "Número de oficiales de policía a nivel estatal."
          },
          {
            "column_name": "hombres",
            "data_type": "INT",
            "is_nullable": false,
            "description": "Número de oficiales de policía de género masculino."
          },
          {
            "column_name": "mujeres",
            "data_type": "INT",
            "is_nullable": false,
            "description": "Número de oficiales de policía de género femenino."
          },
          {
            "column_name": "oficiales_por_habitante",
            "data_type": "FLOAT",
            "is_nullable": true,
            "description": "Ratio de oficiales por cada 100,000 habitantes en la entidad."
          },
          {
            "column_name": "fecha_corte",
            "data_type": "DATE",
            "is_nullable": false,
            "description": "La fecha de corte de los datos, fijada en 2025-01-01 para este dataset hipotético."
          }
        ]
      }
    },
    "business_metadata": {
      "business_owner": "Secretaría de Seguridad y Protección Ciudadana (SSPC)",
      "business_definition": "Este dataset contiene estadísticas clave sobre la composición y distribución de las fuerzas policiales en cada una de las 32 entidades federativas de México, proyectadas para el año 2025. Se utiliza para análisis de políticas públicas, asignación de recursos y reportes de seguridad.",
      "business_terms": [
        "Fuerza policial",
        "Seguridad pública",
        "Oficiales",
        "Entidad federativa",
        "Ratio policial"
      ]
    },
    "operational_metadata": {
      "last_updated": "2025-01-01T00:00:00Z",
      "update_frequency": "Anual",
      "data_quality_score": "99%",
      "access_policy": "Acceso público, disponible en portales de datos abiertos del gobierno.",
      "steward": "Equipo de Análisis de Datos de la SSPC"
    }
  }
]
```

## Contacto {.smaller .center background-image="../images/slides-secondary.png"}

Jesús López M.